In [12]:
import adlfs

In [28]:
# use read only token
sas_token = "se=2023-02-27T21%3A29Z&sp=racwdl&sv=2018-11-09&sr=c&skoid=b4d39ccb-ca9c-4e9d-a183-5e0f3ba3e609&sktid=f6b6dd5b-f02f-441a-99a0-162ac5060bd2&skt=2023-02-20T21%3A30%3A01Z&ske=2023-02-27T21%3A29%3A00Z&sks=b&skv=2018-11-09&sig=p0mG2aPIjQaBOhBFGGf%2BnGJLZhsarSAn2zoWfB0zxMA%3D"
tile_names = [tile[-5:] for tile in adlfs.AzureBlobFileSystem(account_name="snowmelt", credential=sas_token, anon=False).ls('snowmelt/eric/MGRS/')]

In [29]:
years = ['allyears_median','allyears_std','allyears_corr_strength','2015_median','2016_median','2017_median','2018_median','2019_median','2020_median','2021_median','2022_median']
UTM_zone = [10,11,12,13]
resolution = 80

In [30]:
for year in years:
    for zone in UTM_zone:
        f = open(f'vrts/filenames/files_{year}_utm{zone}.txt','w')
        tile_names_zone = [s for s in tile_names if f'{zone}' in s]
        for tile in tile_names_zone:
            tif = f"https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/{tile}/runoff_onset_{tile}_{year}_{resolution}m.tif"
            f.write(f'{tif}\n')
        f.close()

In [31]:
for year in years:
    for zone in UTM_zone:
        !gdalbuildvrt -input_file_list $f'vrts/filenames/files_{year}_utm{zone}.txt' $f'vrts/vrt_{year}_utm{zone}.vrt'

0...10...20...30...40...50...60...70...80...90ERROR 1: HTTP error code : 404
Warning 1: Can't open https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/10UEV/runoff_onset_10UEV_allyears_median_80m.tif. Skipping it
...100 - done.
0...10ERROR 1: HTTP error code : 404
Warning 1: Can't open https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/11SLV/runoff_onset_11SLV_allyears_median_80m.tif. Skipping it
ERROR 1: HTTP error code : 404
Warning 1: Can't open https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/11SMA/runoff_onset_11SMA_allyears_median_80m.tif. Skipping it
...20...30ERROR 1: HTTP error code : 404
Warning 1: Can't open https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/11TLE/runoff_onset_11TLE_allyears_median_80m.tif. Skipping it
...40...50...60...70...80..ERROR 1: HTTP error code : 404
Warning 1: Can't open https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/11TQM/runoff_onset_11TQM_allyears_median_80m.tif. Skipping it
.90..ERROR 1: HTTP error code :